# `01_preprocess.ipynb`

## 0.1 - Setup

### 0.1.1 - Package Imports

In [2]:
# imports from python standard library
import re
from pathlib import Path

# data science packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

# huggingface packages

### 0.1.2 - Constants

In [3]:
DATA_DIR = Path("./data")
DATA_DIR_RAW = DATA_DIR / "raw"
DATA_DIR_PROCESSED = DATA_DIR / "processed"

TRANSCRIPT_DATA = DATA_DIR_RAW / "transcript/The-Office-Lines-V4.csv"
EPISODE_DATA = DATA_DIR_RAW / "episodes/the_office_episodes.csv"

PROCESSED_DATA = DATA_DIR_PROCESSED / "script_data_processed.csv"

### 0.1.3 - Options

In [4]:
# set plotting backend for Pandas
pd.set_option('plotting.backend', 'plotly')

## 0.2 Import Data

In [5]:
script_df = pd.read_csv(
    filepath_or_buffer=TRANSCRIPT_DATA,
    header=0,
    usecols=['season', 'episode', 'title', 'scene', 'speaker', 'line']
    )

script_df.sample(20)

,season,episode,title,scene,speaker,line
27699,6,6,Mafia,4284,Andy,Do it.
730,1,4,The Alliance,120,Pam,What does that even mean?
14965,4,5,Launch Party (Parts 1&2),2521,Angela,It should take no time at all if you put the c...
20783,5,9,Frame Toby,3390,Ryan,I--- I would find a way.
23288,5,19,Golden Ticket,3719,Pam,"Dunder Mifflin, this is Pam. Oh hi ,David. No..."
5929,2,17,Dwight's Speech,974,Angela,"The very best of luck to you, Dwight."
28263,6,8,Koi Pond,4346,Michael,Yes.
10360,3,13,Traveling Salesmen,1745,Andy,"Hey Dwight, pass the tardy sauce. Get it Michael?"
3116,2,6,The Fight,498,Michael,"If you're doing anything crazy, give me a shout."
38059,7,16,PDA,5717,Andy,What's up?


In [6]:
episode_df = pd.read_csv(
    filepath_or_buffer=EPISODE_DATA,
    header=0,
    usecols=['season', 'episode_num_in_season', 'directed_by' , 'written_by']
)

episode_df.sample(20)

,season,episode_num_in_season,directed_by,written_by
96,5,25,Steve Carell,Charlie Grandy
138,7,13,David Rogers,Carrie Kemper
116,6,17,Seth Gordon,Daniel Chun
122,6,23,Mindy Kaling,Justin Spitzer
175,8,24,B. J. Novak,B. J. Novak
34,3,7,Tucker Gates,Michael Schur
66,4,14,Jeffrey Blitz,B. J. Novak
184,9,9,Charles McDougall,Robert Padnick
22,2,17,Charles McDougall,Paul Lieberstein
103,6,4,Paul Feig,Greg Daniels & Mindy Kaling


# 1 - EDA

## 1.1 - Speaker Frequency

In [7]:
# no filtering for seasons
print("frequency:\n\n", script_df['speaker'].value_counts(normalize=False).nlargest(10), sep="")
print("\ntotal of frequency (top 10):\t", script_df['speaker'].value_counts(normalize=False).nlargest(10).sum(), sep="")
print("total of frequency (all):  \t", script_df['speaker'].value_counts(normalize=False).sum(), sep="")
print("\n", "-"*20)
print("\nnormalized:\n\n", script_df['speaker'].value_counts(normalize=True).nlargest(10), sep="")
print("\ntotal of normalized:\t", script_df['speaker'].value_counts(normalize=True).nlargest(10).sum(), sep="")

frequency:

Michael    10773
Dwight      6752
Jim         6222
Pam         4973
Andy        3698
Kevin       1535
Angela      1534
Erin        1413
Oscar       1336
Ryan        1182
Name: speaker, dtype: int64

total of frequency (top 10):	39418
total of frequency (all):  	54626

 --------------------

normalized:

Michael    0.197214
Dwight     0.123604
Jim        0.113902
Pam        0.091037
Andy       0.067697
Kevin      0.028100
Angela     0.028082
Erin       0.025867
Oscar      0.024457
Ryan       0.021638
Name: speaker, dtype: float64

total of normalized:	0.7215977739537949


The top 10 speakers account for ~72% of all lines in the show, with Michael having the highest proportion of individual utterances (just shy of 20%).

In [8]:
top_10_speaker_list = script_df['speaker'].value_counts(normalize=True).nlargest(10).index.tolist()
top_10_speaker_df = script_df.loc[script_df['speaker'].isin(top_10_speaker_list)]

top_10_speaker_df = top_10_speaker_df.astype({'season': str})

top_10_groupedby_season = top_10_speaker_df.groupby(by=['season'])['speaker'].value_counts(normalize=True).unstack('speaker')
top_10_groupedby_speaker = top_10_speaker_df.groupby(by=['speaker'])['season'].value_counts(normalize=True).unstack('season')

In [9]:
title = "Distribution of Each Seasons' Speakers"
hlabel = "Frequency of utterances (normalized)"
vlabel = "Season #"
legend_label = "Speaker"

template = 'plotly_white'
colormap = px.colors.qualitative.D3

width = 1000; height = 500; margin = 110

fig = top_10_groupedby_season.plot.barh(color_discrete_sequence=colormap)

fig.update_layout(
    template=template,
    title=dict(text=title, font=dict(size=20)),
    xaxis_title=dict(text=hlabel, font=dict(size=16)),
    yaxis_title=dict(text=vlabel, font=dict(size=16)),
    legend_title=dict(text=legend_label, font=dict(size=16)),
    margin=dict(l=margin, r=margin, t=margin, b=margin),
    width=width, 
    height=height,
    font=dict(family='Open Sans, Arial', color='black')
)

fig.show()

That's one view of per-season speakers, albeit somewhat challenging to interpret given there are a different number of total utterances per season.

What about each character's speaking per season:

In [10]:
title = "Distribution of Each Speaker's Lines Across Seasons"
hlabel = "Frequency of utterances (normalized)"
vlabel = "Speaker (Limited to Top 10 Speakers)"
legend_label = "Season #"

template = 'plotly_white'
colormap = px.colors.qualitative.D3

width = 1000; height = 500; margin = 110

fig = top_10_groupedby_speaker.plot.barh(color_discrete_sequence=colormap)

fig.update_layout(
    template=template,
    title=dict(text=title, font=dict(size=20)),
    xaxis_title=dict(text=hlabel, font=dict(size=16)),
    yaxis_title=dict(text=vlabel, font=dict(size=16)),
    legend_title=dict(text=legend_label, font=dict(size=16)),
    margin=dict(l=margin, r=margin, t=margin, b=margin),
    width=width, 
    height=height,
    font=dict(family='Open Sans, Arial', color='black')
)

fig.show()

## 1.2 - Line Length

### 1.2.1 - Segment by Character

In [11]:
# smallest grain - character length
line_length_series = script_df['line'].apply(len)

# plot setup
title = "Length of 'line' - Character Length"
hlabel = "Character Length of field 'line'"
vlabel = "Frequency (log scale)"

template = 'plotly_white'
colormap = px.colors.qualitative.D3

width = 1000; height = 500; margin = 110

fig = px.histogram(line_length_series,
                   #nbins=100,
                   log_y=True,
                   color_discrete_sequence=colormap
                   )

fig.update_layout(
    template=template,
    title=dict(text=title, font=dict(size=20)),
    xaxis_title=dict(text=hlabel, font=dict(size=16)),
    yaxis_title=dict(text=vlabel, font=dict(size=16)),
    #legend_title=dict(text=legend_label, font=dict(size=16)),
    showlegend=False,
    margin=dict(l=margin, r=margin, t=margin, b=margin),
    width=width, 
    height=height,
    font=dict(family='Open Sans, Arial', color='black')
)

fig.show()

### 1.2.2 - Segment by Word

In [12]:
# tokenized into words
#   -> using word_tokenize() from nltk.tokenize
#   -> will be using a HuggingFace tokenizer later
line_length_series = script_df['line'].apply(lambda line: len(word_tokenize(line)))

# plot setup
title = "Length of 'line' - Number of Word Tokens"
hlabel = "Number of Words of field 'line'"
vlabel = "Frequency (log scale)"

template = 'plotly_white'
colormap = px.colors.qualitative.D3

width = 1000; height = 500; margin = 110

fig = px.histogram(line_length_series,
                   #nbins=100,
                   log_y=True,
                   color_discrete_sequence=colormap
                   )

fig.update_layout(
    template=template,
    title=dict(text=title, font=dict(size=20)),
    xaxis_title=dict(text=hlabel, font=dict(size=16)),
    yaxis_title=dict(text=vlabel, font=dict(size=16)),
    #legend_title=dict(text=legend_label, font=dict(size=16)),
    showlegend=False,
    margin=dict(l=margin, r=margin, t=margin, b=margin),
    width=width, 
    height=height,
    font=dict(family='Open Sans, Arial', color='black')
)

fig.show()

### 1.2.3 - Segment by Sentence

In [13]:
# tokenized into sentences
#   -> using sent_tokenize() from nltk.tokenize
line_length_series = script_df['line'].apply(lambda line: len(sent_tokenize(line)))

# plot setup
title = "Length of 'line' - Number of Sentences"
hlabel = "Number of Sentences in field 'line'"
vlabel = "Frequency (log scale)"

template = 'plotly_white'
colormap = px.colors.qualitative.D3

width = 1000; height = 500; margin = 110

fig = px.histogram(line_length_series,
                   #nbins=100,
                   log_y=True,
                   color_discrete_sequence=colormap
                   )

fig.update_layout(
    template=template,
    title=dict(text=title, font=dict(size=20)),
    xaxis_title=dict(text=hlabel, font=dict(size=16)),
    yaxis_title=dict(text=vlabel, font=dict(size=16)),
    #legend_title=dict(text=legend_label, font=dict(size=16)),
    showlegend=False,
    margin=dict(l=margin, r=margin, t=margin, b=margin),
    width=width, 
    height=height,
    font=dict(family='Open Sans, Arial', color='black')
)

fig.show()

## 1.3 - Number of Episodes

According to a [Wikipedia list of episodes](https://en.wikipedia.org/wiki/List_of_The_Office_(American_TV_series)_episodes), there were **201** episodes over the nine seasons of The Office (specifically the U.S. version of the show).

Quickly verify how many episodes are included in our dataset.

In [14]:
script_df.groupby(by=['season'])['episode'].unique()[2]

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22], dtype=int64)

In [15]:
# actual
print("Actual number of episodes in show:\t", "201 episodes\n")

# script_df - containing the lines of dialogue
print("Number of episodes in `script_df`:\t", 
      script_df.groupby(by=['season'])['episode'].unique().apply(len).sum(),
      "episodes")

# episode_df - containing the writer and director credits
print("Number of episodes in `episode_df`:\t", 
      episode_df.groupby(by=['season'])['episode_num_in_season'].unique().apply(len).sum(),
      "episodes")

Actual number of episodes in show:	 201 episodes

Number of episodes in `script_df`:	 186 episodes
Number of episodes in `episode_df`:	 201 episodes


While we won't be able to fill in for the missing episodes' dialogue, we can identify which episodes are missing.

Data for episodes-per-season (within the innermost, enumerated list in the next cell) is taken from:
[https://en.wikipedia.org/wiki/List_of_The_Office_(American_TV_series)_episodes#Series_overview](https://en.wikipedia.org/wiki/List_of_The_Office_(American_TV_series)_episodes#Series_overview)

In [16]:
actual_episode_list = {
    season: set(range(1, num_episodes+1)) 
    for season, num_episodes in enumerate([6, 22, 25, 19, 28, 26, 26, 24, 25], start=1)
}

script_df_episode_list = {
    season: set(script_df.groupby(by=['season'])['episode'].unique()[season])
    for season in range(1,9+1)
}

episode_df_episode_list = {
    season: set(episode_df.groupby(by=['season'])['episode_num_in_season'].unique()[season])
    for season in range(1,9+1)
}

print("Episodes missing from `script_df`:")
for season in range(1, 9+1):
    set_difference = actual_episode_list[season].difference(script_df_episode_list[season])
    print(f">> Season {season}: {sorted(set_difference)}")

print("\n", "-"*20, "\n", sep="")

print("Episodes missing from `episode_df`:")
for season in range(1, 9+1):
    set_difference = actual_episode_list[season].difference(episode_df_episode_list[season])
    print(f">> Season {season}: {sorted(set_difference)}")

Episodes missing from `script_df`:
>> Season 1: []
>> Season 2: []
>> Season 3: [11, 25]
>> Season 4: [2, 4, 6, 8, 19]
>> Season 5: [2, 15]
>> Season 6: [5, 18]
>> Season 7: [12, 26]
>> Season 8: []
>> Season 9: [23, 25]

--------------------

Episodes missing from `episode_df`:
>> Season 1: []
>> Season 2: []
>> Season 3: []
>> Season 4: []
>> Season 5: []
>> Season 6: []
>> Season 7: []
>> Season 8: []
>> Season 9: []


## 1.X - Directors / Writers

In [17]:
episode_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   season                 201 non-null    int64 
 1   episode_num_in_season  201 non-null    int64 
 2   directed_by            201 non-null    object
 3   written_by             201 non-null    object
dtypes: int64(2), object(2)
memory usage: 6.4+ KB


In [18]:
episode_df['directed_by'].value_counts()

Paul Feig            17
Ken Kwapis           15
Randall Einhorn      15
Greg Daniels         14
Jeffrey Blitz        13
David Rogers         10
Ken Whittingham      10
Charles McDougall     8
Matt Sohn             8
Paul Lieberstein      8
Harold Ramis          5
B. J. Novak           5
Tucker Gates          4
Brent Forrester       4
Rainn Wilson          4
John Krasinski        3
Troy Miller           3
Jennifer Celotta      3
Steve Carell          3
Craig Zisk            2
Julian Farino         2
Ed Helms              2
Jason Reitman         2
Mindy Kaling          2
Dennie Gordon         2
Victor Nelli, Jr.     2
Seth Gordon           2
Claire Scanlon        2
Rodman Flender        2
Dean Holland          2
Joss Whedon           2
Bryan Gordon          2
Bryan Cranston        1
Daniel Chun           1
Michael Spiller       1
Brian Baumgartner     1
Kelly Cantley         1
Eric Appel            1
Lee Kirk              1
Jon Favreau           1
Charlie Grandy        1
Amy Heckerling  

In [19]:
episode_df['written_by'].value_counts()

Mindy Kaling                                         21
Paul Lieberstein                                     16
B. J. Novak                                          15
Greg Daniels                                         11
Jennifer Celotta                                     10
Brent Forrester                                      10
Justin Spitzer                                        9
Michael Schur                                         8
Gene Stupnitsky & Lee Eisenberg                       8
Charlie Grandy                                        8
Lee Eisenberg & Gene Stupnitsky                       6
Aaron Shure                                           6
Daniel Chun                                           6
Owen Ellickson                                        4
Warren Lieberstein & Halsted Sullivan                 4
Robert Padnick                                        4
Carrie Kemper                                         4
Halsted Sullivan & Warren Lieberstein           

# 2 - Preprocess

## 2.1 - Data Cleaning (Pipeline Setup)

Using a pipeline approach for this, with `sklearn.pipeline.Pipeline` and `sklearn.preprocessing.FunctionTransformer` organizing the effort.

In [20]:
data_cleaning_pipeline = Pipeline(steps=[], verbose=True)

### 2.1.1 - `speaker` - Lowercase values

Set all values of `speaker` column to lowercase.

In [21]:
def lowercase_speaker(df: pd.DataFrame) -> pd.DataFrame:
    # copy df to not modify original
    new_df = df.copy()

    new_df['speaker'] = new_df['speaker'].apply(lambda s: s.lower())

    return new_df

In [22]:
# sanity check for lowercase_speaker()
print("[before]\n", script_df['speaker'].value_counts().nlargest(10).index.tolist())

print("[after]\n",lowercase_speaker(script_df)['speaker'].value_counts().nlargest(10).index.tolist())

[before]
 ['Michael', 'Dwight', 'Jim', 'Pam', 'Andy', 'Kevin', 'Angela', 'Erin', 'Oscar', 'Ryan']
[after]
 ['michael', 'dwight', 'jim', 'pam', 'andy', 'kevin', 'angela', 'erin', 'oscar', 'ryan']


In [23]:
data_cleaning_pipeline.steps.append(
    ("lowercase_speaker", 
     FunctionTransformer(lowercase_speaker, kw_args=dict()))
)

### 2.1.2 - `speaker` - Find orphaned lines

Extra attention is given to the `speaker` column as this is going to be the basis of our class label.

Some values for `speaker` contain near-miss values for common speaker names, e.g. `Jim:` for `Jim`.

Test out some regex patterns for dialogue by `Jim` to understand extent:

In [24]:
test_speaker = 'jim'

script_df.loc[
    (script_df['speaker'].str.lower() != test_speaker)
    &
    (script_df['speaker'].str.lower().str.contains(test_speaker))
    , 'speaker'].value_counts()

Jim:                            33
Fake Jim                         6
Jim and Pam                      4
Andy and Jim                     4
Pam and Jim                      2
JIM9334                          2
Michael and Jim                  2
Jim                              2
Jim, Pam, Michael and Dwight     2
Jim & Dwight                     2
Jim, Josh, and Dwight            1
Pam/Jim                          1
Jim & Pam                        1
Dwight, Jim and Michael          1
 Jim                             1
Jim and Dwight                   1
Jim's Dad                        1
"Jim"                            1
Dwight, Erin, Jim & Kevin        1
Jim's voicemail                  1
Name: speaker, dtype: int64

Most common occurrence is "`Jim:`" but also have occurrences of group dialogue (containing commas, "and", "&"), possessive uses ("Jim's voicemail"), and one episode where Jim was speaking with Pam over an instant messaging program as "JIM9334" (highlighted in next cell, below).

In [25]:
script_df.loc[
    script_df['speaker'] == 'JIM9334'
]

,season,episode,title,scene,speaker,line
18756,5,1,Weight Loss (Parts 1&2),3145,JIM9334,Yeah. C'mon we'll meet halfway. You'll be back...
18761,5,1,Weight Loss (Parts 1&2),3145,JIM9334,The rest stop where that soda exploded on me. ...


Try to apply a regex search to recover some of the clearer examples to potentially associate them with "Jim".

Regex pattern chosen will:
 - be applied case-insensitive
 - allow preceding whitespace but otherwise expect "Jim" at start of string
 - disallow "Jim" to be followed by " and", " &", apostrophes, commas, or digits

In [26]:
test_speaker = 'jim'
test_pattern = r"^\s?" + test_speaker + r"(?!\s?and|'|,|\d|\s?&)"

script_df.loc[
    # first condition is to avoid simple/direct matches of "Jim"
    (script_df['speaker'].str.lower() != test_speaker)
    &
    (script_df['speaker'].str.contains(test_pattern, case=False))
    ]

,season,episode,title,scene,speaker,line
21870,5,13,Prince Family Paper,3524,Jim,This close.
24997,5,25,Broke,3955,Jim,You're just out of business?
31120,6,17,The Delivery (Parts 1&2),4755,Jim,Okay. Sshh. I know.
31800,6,21,Happy Hour,4846,Jim:,made a sale.
31809,6,21,Happy Hour,4846,Jim:,. I had a really hard work out this morning.
31823,6,21,Happy Hour,4849,Jim:,w.
31848,6,21,Happy Hour,4855,Jim:,"would love, love, love to go. Only problem is,..."
31850,6,21,Happy Hour,4855,Jim:,od one.
31852,6,21,Happy Hour,4855,Jim:,"solutely, I will do that right now."
31853,6,21,Happy Hour,4856,Jim:,"gotta tell you, this baby is amazing. She gets..."


It appears the "Happy Hour" episode from Season 6 may contain some errors in the dialogue. We may not want to include "Jim:" after all, to avoid adding noise to our data.

Try the same regex approach using Dwight to see how transferrable the approach is:

In [27]:
test_speaker = 'dwight'
test_pattern = r"^\s?" + test_speaker + r"(?!\s?and|'|,|\d|\s?&|:)"

script_df.loc[
    # first condition is to avoid simple/direct matches of test speaker
    (script_df['speaker'].str.lower() != test_speaker)
    &
    (script_df['speaker'].str.contains(test_pattern, case=False))
    ]

,season,episode,title,scene,speaker,line
8751,3,7,Branch Closing,1498,Dwight,"When you become close with someone, you develo..."
14746,4,5,Launch Party (Parts 1&2),2493,DwightKSchrute,You tell me.
14749,4,5,Launch Party (Parts 1&2),2493,DwightKSchrute,How do I know this isn't Jim?
14809,4,5,Launch Party (Parts 1&2),2499,DwightKSchrute,Here's a suggestion computer. I assume you re...
20103,5,7,Customer Survey,3321,Dwight,Customer service might be monitoring this conv...
32185,6,22,Secretary's Day,4904,Dwight.,"Hello, Pam."


We again see an instant messager screenname ("DwightKSchrute") appear, more problematic dialogue from S6E21 "Happy Hour", and otherwise ~3 lines that could be "saved".

For a Dwight/Not Dwight classifier, these three lines would represent $\frac{3}{(6,752+3)}$ or 0.044\% of Dwight's individual utterances and $\frac{3}{39,418+3}$ or 0.008\% of the top 10 character's utterances.

In [28]:
print(f"percent of Dwight's utterances: {3 / (6752+3) * 100:0.3f} %")
print(f"percent of top 10's utterances: {3 / (39418+3) * 100:0.3f} %")

percent of Dwight's utterances: 0.044 %
percent of top 10's utterances: 0.008 %


For this cleaning step, we'll add the colon to the exclusionary regex. Later, we'll examine S06E21 to see if its dialogue can be used at all.

In [29]:
def connect_orphaned_lines(df: pd.DataFrame, speaker_list: list[str]) -> pd.DataFrame:
    # copy incoming df to not modify original
    new_df = df.copy()
    
    for speaker in speaker_list:
        match_pattern = r"^\s?" + speaker + r"(?!\s?and|'|,|\d|\s?&|:)"

        new_df.loc[
            # first condition is to avoid simple/direct matches of test speaker
            (new_df['speaker'].str.lower() != speaker.lower())
            &
            # second condition matches the regex pattern 
            (new_df['speaker'].str.contains(match_pattern, case=False)),
            'speaker'] = speaker
    
    return new_df

In [30]:
# sanity check of connect_orphaned_lines()
test_speaker = "dwight"
test_pattern = r"^\s?" + test_speaker + r"(?!\s?and|'|,|\d|\s?&|:)"

print("[before] utterances by 'dwight' or 'Dwight':", 
      script_df.loc[(script_df.speaker.str.lower() == 'dwight'), 'speaker'].count())

sanity_df = connect_orphaned_lines(script_df, [test_speaker])

print("[after]  utterances by 'dwight' or 'Dwight':",
      sanity_df.loc[(sanity_df.speaker.str.lower() == 'dwight'), 'speaker'].count())

[before] utterances by 'dwight' or 'Dwight': 6752
[after]  utterances by 'dwight' or 'Dwight': 6758


In [31]:
top_10_speaker_list = script_df['speaker'].value_counts(normalize=True).nlargest(10).index.tolist()

data_cleaning_pipeline.steps.append(
    ("connect_orphaned_lines_top10", 
     FunctionTransformer(connect_orphaned_lines, kw_args=dict(speaker_list=top_10_speaker_list)))
)

### 2.1.3 - `season` / `episode` - Problematic Episodes

#### 2.1.3.1 - S06E21 "Happy Hour"

The text of this episode appears to have an issue in the dataset CSV file.

Here's an excerpt from the raw CSV file text showing a few lines of "good" data followed by the first few lines of this episode's data:

```
season,episode,title,scene,speaker,line,
...
6,20,New Leads,4844,Dwight,We have an awesome bean bag chair that's perfect for the break room. ,
6,20,New Leads,4844,Phyllis,Yuck. I'm not going to sit on that disgusting seat.,
6,20,New Leads,4844,Dwight,"Yeah, damn right you're not. 'Cause it's for me and Michael only ",
6,20,New Leads,4845,Andy, It's freezing out here.,
6,20,New Leads,4845,Erin, Go on. I have warm blood. ,
6,20,New Leads,4845,Andy,"Oh wow, thank you. You're the nicest person I've ever met. ",
6,21,Happy Hour,4846,Dwight: ,ichael grunts and strains while doing push-ups] Breathe. Work your core. Come on.,
6,21,Happy Hour,4846,Michael: ,w many is that?,
6,21,Happy Hour,4846,Dwight: ,"t counting the last one, 25.",
6,21,Happy Hour,4846,Michael: ,unt the last one.,
6,21,Happy Hour,4846,Dwight: ,", 25 and one girl push-up!",
6,21,Happy Hour,4846,Michael: ,", new record!",
...
```

The remainder of the dialogue from the "Happy Hour" episode shows the same types of aberrations:  
 - Speaker name ends with a colon "`:`" character followed by a space
 - The first character(s) of text from the line appear to be cut off
 - Some descriptive text, describing a non-spoken detail surrounding the line and enclosed in square brackets (`[]`), remains in the lines.
    - For example: "`ichael grunts and strains while doing push-ups]`"

The data for these lines are still distinguishable based on the numeric fields for `season` and `episode`, so they could be easily and accurately filtered out of our dataset.

In [32]:
def remove_episode(df: pd.DataFrame, season: int, episode: int) -> pd.DataFrame:
    """Removes the episode's rows from the overall dataframe."""
    mask_index = df.loc[
        (
            (df['season'] == season) 
            & 
            (df['episode'] == episode)
        )
    ].index

    return df.drop(index=mask_index)

In [33]:
# sanity check for remove_episode()
print("[before] episodes in season 6:\n", script_df.loc[script_df['season'] == 6, 'episode'].unique())

sanity_df = remove_episode(script_df, season=6, episode=21)

print("\n[after] episodes in season 6:\n", sanity_df.loc[script_df['season'] == 6, 'episode'].unique())

[before] episodes in season 6:
 [ 1  2  3  4  6  7  8  9 10 11 12 13 14 15 16 17 19 20 21 22 23 24 25 26]

[after] episodes in season 6:
 [ 1  2  3  4  6  7  8  9 10 11 12 13 14 15 16 17 19 20 22 23 24 25 26]


For now we're only removing this one episode, so adding this to the data cleaning pipeline.

In [34]:
data_cleaning_pipeline.steps.append(
    ("remove_episode_s6e21", 
     FunctionTransformer(remove_episode, kw_args=dict(season=6, episode=21)))
)

## 2.2 - Data Cleaning (Pipeline Execution)

In [35]:
# note: no 'fit' functionality, but with fit_transform and verbose=True this
#   prints some nice output of the pipeline steps
script_df_cleaned = data_cleaning_pipeline.fit_transform(script_df)

script_df_cleaned.head()

[Pipeline] . (step 1 of 3) Processing lowercase_speaker, total=   0.0s
[Pipeline]  (step 2 of 3) Processing connect_orphaned_lines_top10, total=   0.2s
[Pipeline]  (step 3 of 3) Processing remove_episode_s6e21, total=   0.0s


,season,episode,title,scene,speaker,line
0,1,1,Pilot,1,michael,All right Jim. Your quarterlies look very good...
1,1,1,Pilot,1,jim,"Oh, I told you. I couldn't close it. So..."
2,1,1,Pilot,1,michael,So you've come to the master for guidance? Is ...
3,1,1,Pilot,1,jim,"Actually, you called me in here, but yeah."
4,1,1,Pilot,1,michael,"All right. Well, let me show you how it's done."


Check our results:

In [36]:
# looking for all lowercase values and dwight=6752
script_df_cleaned['speaker'].value_counts().nlargest(10)

michael    10774
dwight      6752
jim         6223
pam         4973
andy        3698
kevin       1535
angela      1534
erin        1413
oscar       1336
ryan        1182
Name: speaker, dtype: int64

In [37]:
# looking for no "21" value in this list
script_df_cleaned.loc[script_df['season'] == 6, 'episode'].unique()

array([ 1,  2,  3,  4,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 19,
       20, 22, 23, 24, 25, 26], dtype=int64)

## 2.3 - Episode Writer / Episode Director

### 2.3.1 - Merge `episode_df` into `script_df`

In [38]:
script_df_enriched: pd.DataFrame = script_df_cleaned.merge(episode_df, 
                        how='left',
                        left_on=['season', 'episode'],
                        right_on=['season', 'episode_num_in_season'],
                        #suffixes=[None, '_y']
                    )

# drop extra "episode_num_in_season" column
script_df_enriched.drop(columns=['episode_num_in_season'], inplace=True)

In [39]:
# check results - check episodes written by "B. J. Novak"
#   Source: https://en.wikipedia.org/wiki/List_of_The_Office_(American_TV_series)_episodes
script_df_enriched.loc[
        script_df_enriched['written_by'] == "B. J. Novak"
    ].groupby(by=['season'])['episode'].unique()

# expecting to see:
#   1: 2
#   2: 2, 4, 15
#   3: 5, 20
#   (and more)

season
1           [2]
2    [2, 4, 15]
3       [5, 20]
4       [9, 14]
5      [13, 22]
7       [2, 17]
8       [1, 24]
Name: episode, dtype: object

In [40]:
# check results - check for episode directed by "Bryan Cranston"
#   Source: https://en.wikipedia.org/wiki/Work_Bus
script_df_enriched.loc[
        script_df_enriched['directed_by'] == "Bryan Cranston"
    ].groupby(by=['season'])['episode'].unique()

# expecting to see:
#   Season 9, Episode 4

season
9    [4]
Name: episode, dtype: object

### 2.3.2 - Tokenize (by person)

We saw in EDA that:
 - Each episode is listed with **one** Director credit (this makes intuitive sense). Because of this, we won't need to separate the Director column into separate names.
 - Some episodes are listed with multiple Writer credits, using ampersands or the word "and" (or a mix of the two) to separate each Writer's name.
 - There were no episodes with more than **three** Writers credited.

We'll divide up Writers into three new columns: `writer`, `writer2`, and `writer3`.

In [41]:
# create function to map across dataframe
def parse_writers(df_row: pd.Series) -> pd.Series:
    split_pattern = r"(?:\s+and\s+|\s+&\s+)"
    writers = re.split(split_pattern, df_row['written_by'])

    if (len(writers) >= 1):
        df_row['writer1'] = writers[0]

    if (len(writers) >= 2):
        df_row['writer2'] = writers[1]

    if (len(writers) == 3):
        df_row['writer3'] = writers[2]

    return df_row

In [43]:
# create three blank columns
script_df_enriched['writer1'] = None
script_df_enriched['writer2'] = None
script_df_enriched['writer3'] = None

# map the parsing function to each row
script_df_enriched = script_df_enriched.apply(parse_writers, axis='columns')

# check results
pd.concat([
    script_df_enriched.loc[script_df_enriched['writer2'].isna()].sample(2, random_state=42),
    script_df_enriched.loc[script_df_enriched['writer2'].notna()].sample(2, random_state=42),
    script_df_enriched.loc[script_df_enriched['writer3'].notna()].sample(2, random_state=42)
    ])

,season,episode,title,scene,speaker,line,directed_by,written_by,writer1,writer2,writer3
31908,6,22,Secretary's Day,4917,pam,Ooh. Ah-ah-ah. Ha-ha.,Steve Carell,Mindy Kaling,Mindy Kaling,None,None
51041,9,14,Vandalism,7662,pam,"Oh, no. Um, I, I'm embarrassed. It's stupid.",Lee Kirk,Owen Ellickson,Owen Ellickson,None,None
32511,6,24,The Cover-Up,5002,dwight,Oh. Oh. Oh. One thing you need to know about...,Rainn Wilson,Gene Stupnitsky & Lee Eisenberg,Gene Stupnitsky,Lee Eisenberg,None
17750,4,17,Job Fair,2963,pam,"Just checking in, how's it going?",Tucker Gates,Lee Eisenberg & Gene Stupnitsky,Lee Eisenberg,Gene Stupnitsky,None
158,1,1,Pilot,32,dwight,Yeah.,Ken Kwapis,Ricky Gervais & Stephen Merchant and Greg Daniels,Ricky Gervais,Stephen Merchant,Greg Daniels
10594,3,14,The Return,1798,jim,Is that you singing?,Greg Daniels,Lee Eisenberg & Gene Stupnitsky & Michael Schur,Lee Eisenberg,Gene Stupnitsky,Michael Schur


# 3 - Export Dataset

In [44]:
if (not DATA_DIR_PROCESSED.exists()):
    DATA_DIR_PROCESSED.mkdir()

script_df_enriched.to_csv(
    PROCESSED_DATA,
    header=True,
    encoding='utf-8'
)